# 実行環境の作成

## GCP認証

下記コードでGCPに接続

In [4]:
from google.colab import auth
auth.authenticate_user()

# estatの統計情報を取得

メタ情報を取得する関数を作成

In [6]:
import urllib.parse
import urllib.request
import json

def get_estat_meta(statsDataId):
  params ={
      'statsDataId': statsDataId,
      'appId': '724e5b90772a3e9289f41a253e4e7e32438f4fff',
  }

  url = 'http://api.e-stat.go.jp/rest/3.0/app/json/getMetaInfo?'
  url += urllib.parse.urlencode(params)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

In [7]:
meta = get_estat_meta('0000020211')
print(meta)

{'GET_META_INFO': {'RESULT': {'STATUS': 0, 'ERROR_MSG': '正常に終了しました。', 'DATE': '2022-12-28T21:25:33.854+09:00'}, 'PARAMETER': {'LANG': 'J', 'STATS_DATA_ID': '0000020211', 'DATA_FORMAT': 'J'}, 'METADATA_INF': {'TABLE_INF': {'@id': '0000020211', 'STAT_NAME': {'@code': '00200502', '$': '社会・人口統計体系'}, 'GOV_ORG': {'@code': '00200', '$': '総務省'}, 'STATISTICS_NAME': '市区町村データ 基礎データ（廃置分合処理済）', 'TITLE': {'@no': '0000020211', '$': 'Ｋ\u3000安全'}, 'CYCLE': '年度次', 'SURVEY_DATE': 0, 'OPEN_DATE': '2022-06-21', 'SMALL_AREA': 0, 'COLLECT_AREA': '市区町村', 'MAIN_CATEGORY': {'@code': '99', '$': 'その他'}, 'SUB_CATEGORY': {'@code': '99', '$': 'その他'}, 'OVERALL_TOTAL_NUMBER': 112933, 'UPDATED_DATE': '2022-06-21', 'STATISTICS_NAME_SPEC': {'TABULATION_CATEGORY': '市区町村データ', 'TABULATION_SUB_CATEGORY1': '基礎データ（廃置分合処理済）'}, 'DESCRIPTION': {'TABULATION_CATEGORY_EXPLANATION': '社会・人口統計体系の市区町村ごとに集計したデータを提供します。', 'TABULATION_SUB_CATEGORY_EXPLANATION1': 'データは、2021年3月31日時点の市区町村で整備しています。'}, 'TITLE_SPEC': {'TABLE_NAME': 'Ｋ\u3000安全'}}

## DataFrameの作成

メタ情報をDataFrameに整形する

In [1]:
import pandas as pd

def df_meta(statsDataId):
  # メタ情報の取得
  meta= get_estat_meta(statsDataId)

  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']
  cat01 = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']
  
  # 都道府県 or 市区町村
  COLLECT_AREA = meta['GET_META_INFO']['METADATA_INF']['TABLE_INF']['COLLECT_AREA']
  governmentType = 'prefecture' if COLLECT_AREA == '全国' else 'city'

  # DataFrame
  df = pd.json_normalize(cat01)[['@code','@name','@unit']]
  
  # 列名の変更
  columns = {'@code':'categoryCode', '@name':'categoryName', '@unit':'unit'}
  df= df.rename(columns=columns)

  # categoryNameから不要な情報（categoryCode）を削除
  df['categoryName'] = df.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)
  
  # 並べ替え
  df = df.reindex(columns=['categoryCode','categoryName','unit'])
  df['statsDataId'] = statsDataId
  df['statsDataName'] = meta['GET_META_INFO']['METADATA_INF']['TABLE_INF']['STAT_NAME']['$']
  df['governmentType'] = governmentType

  return df

単体テスト

In [8]:
df = df_meta('0000010102')
df

,categoryCode,categoryName,unit,statsDataId,statsDataName,governmentType
0,B1101,総面積（北方地域及び竹島を除く）,ｈａ,0000010102,社会・人口統計体系,prefecture
1,B1102,総面積（北方地域及び竹島を含む）,ｈａ,0000010102,社会・人口統計体系,prefecture
2,B1103,可住地面積,ｈａ,0000010102,社会・人口統計体系,prefecture
3,B1104,主要湖沼面積,ｈａ,0000010102,社会・人口統計体系,prefecture
4,B1105,林野面積,ｈａ,0000010102,社会・人口統計体系,prefecture
5,B1106,森林面積,ｈａ,0000010102,社会・人口統計体系,prefecture
6,B1107,森林以外の草生地面積,ｈａ,0000010102,社会・人口統計体系,prefecture
7,B1108,自然環境保全地域面積,ｈａ,0000010102,社会・人口統計体系,prefecture
8,B1201,評価総地積（課税対象土地）,ｍ2,0000010102,社会・人口統計体系,prefecture
9,B120101,評価総地積（田）,ｍ2,0000010102,社会・人口統計体系,prefecture


一括処理

In [9]:
def df_metas(data):
  df = pd.DataFrame(index=[], columns=[])
  for d in data:
    df1 = df_meta(d)
    df = pd.concat([df,df1],ignore_index=True)

  return df

In [11]:
data = ['0000010101', '0000010102', '0000010103', '0000010104', '0000010105', '0000010106', '0000010107', 
        '0000010108', '0000010109', '0000010110', '0000010111', '0000010112', '0000010113',
        '0000020201', '0000020202', '0000020203', '0000020204', '0000020205', '0000020206', 
        '0000020207', '0000020208', '0000020209', '0000020210', '0000020211']

df = df_metas(data)
df.head()

,categoryCode,categoryName,unit,statsDataId,statsDataName,governmentType
0,A1101,総人口,人,0000010101,社会・人口統計体系,prefecture
1,A110101,総人口（男）,人,0000010101,社会・人口統計体系,prefecture
2,A110102,総人口（女）,人,0000010101,社会・人口統計体系,prefecture
3,A1102,日本人人口,人,0000010101,社会・人口統計体系,prefecture
4,A110201,日本人人口（男）,人,0000010101,社会・人口統計体系,prefecture


In [12]:
from google.cloud import bigquery

def save_gbq(df):
  project_id='primal-buttress-342908'
  dataset_id = 'estat_config'
  table_id = 'categories'
  
  # スキーマ
  job_config = bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE')

  # bigqueryへ保存
  client = bigquery.Client(project=project_id)
  job = client.load_table_from_dataframe(df, f'{dataset_id}.{table_id}', job_config=job_config).result()

  # 確認メッセージ  
  table = client.get_table(f'{dataset_id}.{table_id}') 
  print("{} 件のデータを「{}」に保存しました。".format(table.num_rows, table_id))

  return 

In [14]:
df = df_metas(data)
save_gbq(df)

5257 件のデータを「categories」に保存しました。
